In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd                 # dataframes


import datetime   #para normalizar fechas
import time  #para obtener fechas
from dateutil.rrule import rrule, DAILY , MONTHLY   #hacer los calendarios de iteración

##### Se establece el rango de fechas

In [35]:
def fechas():
    ''' 
    Función:Definimos las fechas de inicio y fin
    Entrada:
    Salida: Fechas a cargar en db
    '''
    #Fecha inicio
    a = datetime.date(2022, 1, 1)
    #Fecha fin
    b = datetime.date(2022, 12, 31)
    return a,b

##### Se establece la funcion de generaciòn de url por fecha

In [29]:
def get_url_chile(dat):
    year=dat.strftime("%Y")
    month=dat.strftime("%m")
    day=dat.strftime('%Y%m%d')

    url=f"https://www.sismologia.cl/sismicidad/catalogo/{year}/{month}/{day}.html"
    urlx = requests.get (url)
    soup = BeautifulSoup(urlx.content,"html.parser")
    rows = soup.find("table", attrs={"class":"sismologia detalle"}).find_all("tr")
    return  rows 

##### Se realiza el web scraping dia por dia

In [32]:
a,b=fechas()
data=[]
for dat in rrule(DAILY,dtstart=a,until=b):
    rows=get_url_chile(dat)
    
    for row in rows:
        cells = row.find_all("td")
        row_data = []
        for cell in cells:
            row_data.append(cell.text.strip())
        data.append(row_data)
    

##### Se genera la tabla a revisar

In [33]:
df =pd.DataFrame(data, columns=["Fecha Local / Lugar","Fecha UTC","Latitud / Longitud","Profundidad","Magnitud"])

In [36]:
df.head(4)

,Fecha Local / Lugar,Fecha UTC,Latitud / Longitud,Profundidad,Magnitud
0,None,None,None,None,None
1,2022-01-01 19:57:1837 km al SO de Mina Collahuasi,2022-01-01 22:57:18,-21.098 -68.863,109 km,2.7 Ml
2,2022-01-01 18:53:3018 km al N de Ovalle,2022-01-01 21:53:30,-30.443 -71.236,24 km,2.7 Ml
3,2022-01-01 18:26:14100 km al SE de Socaire,2022-01-01 21:26:14,-24.323 -67.312,175 km,3.3 Ml


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7723 entries, 0 to 7722
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Fecha Local / Lugar  7358 non-null   object
 1   Fecha UTC            7358 non-null   object
 2   Latitud / Longitud   7358 non-null   object
 3   Profundidad          7358 non-null   object
 4   Magnitud             7358 non-null   object
dtypes: object(5)
memory usage: 301.8+ KB
